# Vectorstores and Embeddings

Recall the overall workflow for retrieval augmented generation (RAG):

In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
load_dotenv(dotenv_path="/Users/pulkitaggarwal/LangChain2/Gemini_Key.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("Hello Gemini")
print(response.text)

Hello there!  How can I help you today?



In [2]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [4]:
splits = text_splitter.split_documents(docs)

In [5]:
len(splits)

208

## Embeddings

Let's take our splits and embed them.

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [7]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [8]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [9]:
import numpy as np

In [10]:
np.dot(embedding1, embedding2)

np.float64(0.9297800180090847)

In [11]:
np.dot(embedding1, embedding3)

np.float64(0.6170382633502922)

In [12]:
np.dot(embedding2, embedding3)

np.float64(0.600990672581756)

### Vectorstores

In [13]:
pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 30.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 30.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 28.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 16.2 MB/s eta 0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=7c021d063905ba759b857cb22d14fefb9010238b5ea9686566b22fe62bded592
  Stored in directory: /Users/pulkitaggarwal/Library/Caches/pip

In [14]:
from langchain.vectorstores import Chroma

In [15]:
persist_directory = 'docs/chroma/'

In [16]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [17]:
print(vectordb._collection.count())

208


### Similarity Search

In [18]:
question = "is there an email i can ask for help"

In [19]:
docs = vectordb.similarity_search(question,k=3)

In [20]:
len(docs)

3

In [21]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

In [22]:
vectordb.persist()

/var/folders/wd/hxgvmlms6j39627ykffdygcm0000gn/T/ipykernel_80546/3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


## Failure modes

This seems great, and basic similarity search will get you 80% of the way there very easily. 

But there are some failure modes that can creep up. 

Here are some edge cases that can arise - we'll fix them in the next class.

In [23]:
question = "what did they say about matlab?"

In [24]:
docs = vectordb.similarity_search(question,k=5)

Notice that we're getting duplicate chunks (because of the duplicate `MachineLearning-Lecture01.pdf` in the index).

Semantic search fetches all similar documents, but does not enforce diversity.

`docs[0]` and `docs[1]` are indentical.

In [25]:
docs[0]

Document(metadata={'moddate': '2008-07-11T11:25:23-07:00', 'total_pages': 22, 'author': '', 'page_label': '9', 'source': '/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture01.pdf', 'title': '', 'page': 8, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creationdate': '2008-07-11T11:25:23-07:00', 'creator': 'PScript5.dll Version 5.2.2'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if y

In [26]:
docs[1]

Document(metadata={'page': 8, 'source': '/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture01.pdf', 'page_label': '9', 'total_pages': 22, 'creator': 'PScript5.dll Version 5.2.2', 'author': '', 'title': '', 'creationdate': '2008-07-11T11:25:23-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'moddate': '2008-07-11T11:25:23-07:00'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if y

We can see a new failure mode.

The question below asks a question about the third lecture, but includes results from other lectures as well.

In [27]:
question = "what did they say about regression in the third lecture?"

In [28]:
docs = vectordb.similarity_search(question,k=5)

In [29]:
for doc in docs:
    print(doc.metadata)

{'moddate': '2008-07-11T11:25:03-07:00', 'title': '', 'source': '/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture03.pdf', 'total_pages': 16, 'page': 14, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creationdate': '2008-07-11T11:25:03-07:00', 'page_label': '15', 'creator': 'PScript5.dll Version 5.2.2', 'author': ''}
{'creationdate': '2008-07-11T11:25:03-07:00', 'source': '/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture03.pdf', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'total_pages': 16, 'title': '', 'page_label': '14', 'page': 13, 'moddate': '2008-07-11T11:25:03-07:00', 'author': '', 'creator': 'PScript5.dll Version 5.2.2'}
{'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'source': '/Users/pulkitaggarwal/LangChain2/MachineLearning-Lecture03.pdf', 'author': '', 'title': '', 'creator': 'PScript5.dll Version 5.2.2', 'total_pages': 16, 'creationdate': '2008-07-11T11:25:03-07:00', 'moddate': '2008-07-11T11:25:03-07:00', 'page_label': '1', 'page': 0}
{'author': '

In [30]:
print(docs[4].page_content)

MachineLearning-Lecture03  
Instructor (Andrew Ng):Okay. Good morning and welcome back to the third lecture of 
this class. So here’s what I want to do today, and some of the topics I do today may seem 
a little bit like I’m jumping, sort of, from topic to topic, but here’s, sort of, the outline for 
today and the illogical flow of ideas. In the last lecture, we talked about linear regression 
and today I want to talk about sort of an adaptation of that called locally weighted 
regression. It’s very a popular algorithm that’s actually one of my former mentors 
probably favorite machine learning algorithm.  
We’ll then talk about a probable second interpretation of linear regression and use that to 
move onto our first classification algorithm, which is logistic regression; take a brief 
digression to tell you about something called the perceptron algorithm, which is 
something we’ll come back to, again, later this quarter; and time allowing I hope to get to 
Newton’s method, which is a